In [1]:
import os
import glob
from PIL import Image
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt
from pathlib import Path

In [2]:
data_dir = '../Data/0325updated.task1train(626p)-20251119T175414Z-1-001/0325updated.task1train(626p)'

print("="*60)
print("IMAGE DATASET ANALYSIS")
print("="*60)

# Find all image files
image_extensions = ['*.jpg', '*.jpeg', '*.png', '*.bmp', '*.tiff', '*.gif']
image_files = []
for ext in image_extensions:
    image_files.extend(glob.glob(os.path.join(data_dir, ext)))

print(f"\nTotal images found: {len(image_files)}")

IMAGE DATASET ANALYSIS

Total images found: 712


In [3]:
if len(image_files) == 0:
    print("No images found! Please check the directory path.")
else:
    # Analyze images
    print("\nAnalyzing images...")
    
    widths = []
    heights = []
    aspect_ratios = []
    modes = []
    file_sizes = []
    channels = []
    
    corrupted_files = []
    
    for img_path in image_files:
        try:
            # Get file size
            file_size = os.path.getsize(img_path) / 1024  # KB
            file_sizes.append(file_size)
            
            # Open and analyze image
            img = Image.open(img_path)
            widths.append(img.width)
            heights.append(img.height)
            aspect_ratios.append(img.width / img.height)
            modes.append(img.mode)
            
            # Count channels
            if img.mode == 'RGB':
                channels.append(3)
            elif img.mode == 'RGBA':
                channels.append(4)
            elif img.mode == 'L':
                channels.append(1)
            else:
                channels.append(len(img.getbands()))
                
        except Exception as e:
            corrupted_files.append((os.path.basename(img_path), str(e)))
    
    # Statistics
    print("\n" + "="*60)
    print("IMAGE STATISTICS")
    print("="*60)
    
    print(f"\n📐 Dimensions:")
    print(f"   Width  - Min: {min(widths):4d}px | Max: {max(widths):4d}px | Avg: {np.mean(widths):.1f}px")
    print(f"   Height - Min: {min(heights):4d}px | Max: {max(heights):4d}px | Avg: {np.mean(heights):.1f}px")
    
    print(f"\n📊 Aspect Ratios:")
    print(f"   Min: {min(aspect_ratios):.3f} | Max: {max(aspect_ratios):.3f} | Avg: {np.mean(aspect_ratios):.3f}")
    
    # Check if dimensions are consistent
    unique_dims = set(zip(widths, heights))
    if len(unique_dims) == 1:
        print(f"\n✓ All images have SAME dimensions: {widths[0]}x{heights[0]}")
    else:
        print(f"\n⚠ Images have DIFFERENT dimensions ({len(unique_dims)} unique sizes)")
        print(f"   Most common sizes:")
        dim_counter = Counter(zip(widths, heights))
        for (w, h), count in dim_counter.most_common(5):
            print(f"     {w}x{h}: {count} images ({count/len(image_files)*100:.1f}%)")
    
    print(f"\n🎨 Color Modes:")
    mode_counter = Counter(modes)
    for mode, count in mode_counter.most_common():
        print(f"   {mode}: {count} images ({count/len(image_files)*100:.1f}%)")
    
    print(f"\n📦 File Sizes:")
    print(f"   Min: {min(file_sizes):.1f} KB | Max: {max(file_sizes):.1f} KB | Avg: {np.mean(file_sizes):.1f} KB")
    
    if corrupted_files:
        print(f"\n⚠ Corrupted/Problematic Images: {len(corrupted_files)}")
        for fname, error in corrupted_files[:5]:
            print(f"   - {fname}: {error}")
    else:
        print(f"\n✓ All images loaded successfully!")
    
    # Recommendations
    print("\n" + "="*60)
    print("PREPROCESSING RECOMMENDATIONS")
    print("="*60)
    
    needs_resize = len(unique_dims) > 1
    needs_normalization = True
    needs_grayscale_conversion = 'RGB' in modes or 'RGBA' in modes
    
    print("\n🔧 Required preprocessing steps:")
    
    step = 1
    if needs_resize:
        print(f"\n{step}. RESIZE IMAGES ⚠️")
        print(f"   Issue: {len(unique_dims)} different image sizes")
        print(f"   Action: Resize all images to consistent dimensions")
        print(f"   Suggested size: {max(set(widths), key=widths.count)}x{max(set(heights), key=heights.count)} (most common)")
        step += 1
    else:
        print(f"\n{step}. RESIZE IMAGES ✓")
        print(f"   All images already have consistent size: {widths[0]}x{heights[0]}")
        step += 1
    
    print(f"\n{step}. NORMALIZATION")
    print(f"   Action: Normalize pixel values to [0, 1] or [-1, 1]")
    print(f"   Current: Pixel values likely in [0, 255]")
    step += 1
    
    if needs_grayscale_conversion:
        print(f"\n{step}. COLOR MODE CONVERSION (Optional)")
        print(f"   Current modes: {', '.join(mode_counter.keys())}")
        print(f"   Consider: Converting to grayscale if color not needed")
        print(f"   Benefits: Reduces model complexity, faster training")
        step += 1
    
    print(f"\n{step}. DATA AUGMENTATION (Recommended)")
    print(f"   Techniques to consider:")
    print(f"   - Random rotation (±5-10°)")
    print(f"   - Random brightness/contrast adjustment")
    print(f"   - Random scaling (90-110%)")
    print(f"   - Elastic distortions (for text/OCR tasks)")
    print(f"   Benefits: Improves model generalization")
    step += 1
    
    print(f"\n{step}. VERIFY IMAGE-TEXT PAIRS")
    print(f"   Action: Ensure each image has corresponding text file")
    print(f"   Images: {len(image_files)}")
    
    # Check for matching text files
    txt_files = glob.glob(os.path.join(data_dir, '*.txt'))
    print(f"   Text files: {len(txt_files)}")
    
    if len(image_files) != len(txt_files):
        print(f"   ⚠️ MISMATCH: {abs(len(image_files) - len(txt_files))} files difference")
    else:
        print(f"   ✓ Same number of images and text files")



Analyzing images...

IMAGE STATISTICS

📐 Dimensions:
   Width  - Min:  436px | Max: 4961px | Avg: 1269.9px
   Height - Min:  605px | Max: 7016px | Avg: 2283.6px

📊 Aspect Ratios:
   Min: 0.263 | Max: 0.971 | Avg: 0.511

⚠ Images have DIFFERENT dimensions (500 unique sizes)
   Most common sizes:
     4961x7016: 74 images (10.4%)
     1080x1527: 17 images (2.4%)
     1080x1528: 13 images (1.8%)
     793x1373: 5 images (0.7%)
     619x1475: 4 images (0.6%)

🎨 Color Modes:
   RGB: 708 images (99.4%)
   L: 4 images (0.6%)

📦 File Sizes:
   Min: 57.8 KB | Max: 3540.2 KB | Avg: 535.5 KB

✓ All images loaded successfully!

PREPROCESSING RECOMMENDATIONS

🔧 Required preprocessing steps:

1. RESIZE IMAGES ⚠️
   Issue: 500 different image sizes
   Action: Resize all images to consistent dimensions
   Suggested size: 4961x7016 (most common)

2. NORMALIZATION
   Action: Normalize pixel values to [0, 1] or [-1, 1]
   Current: Pixel values likely in [0, 255]

3. COLOR MODE CONVERSION (Optional)
   Cu

In [4]:
# Extract basenames (without extensions)
image_basenames = {Path(img).stem: img for img in image_files}
txt_basenames = {Path(txt).stem: txt for txt in txt_files}

In [5]:
# Find images without text
images_without_text = set(image_basenames.keys()) - set(txt_basenames.keys())
print(f"\n📷 Images WITHOUT corresponding text: {len(images_without_text)}")
if images_without_text:
    for basename in sorted(list(images_without_text)[:10]):
        print(f"   - {basename}")
    if len(images_without_text) > 10:
        print(f"   ... and {len(images_without_text) - 10} more")

# Find text without images
text_without_images = set(txt_basenames.keys()) - set(image_basenames.keys())
print(f"\n📝 Text files WITHOUT corresponding images: {len(text_without_images)}")
if text_without_images:
    for basename in sorted(list(text_without_images)[:10]):
        print(f"   - {basename}.txt")
    if len(text_without_images) > 10:
        print(f"   ... and {len(text_without_images) - 10} more")


📷 Images WITHOUT corresponding text: 8
   - X51005433492(1)
   - X51005442384(1)
   - X51005605333(1)
   - X51005676539(1)
   - X51005685355(2)
   - X51005685357(2)
   - X51007339118(1)
   - X51007339647(1)

📝 Text files WITHOUT corresponding images: 131
   - X51005587267(1).txt
   - X51005719917(1).txt
   - X51005806679(1).txt
   - X51006008197(1).txt
   - X51006334699(3).txt
   - X51006556738(1).txt
   - X51006619694(1).txt
   - X51007231338(1).txt
   - X51007339115(1).txt
   - X51007339656(1).txt
   ... and 121 more


In [13]:
# Find matching pairs
matching_pairs = set(image_basenames.keys()) & set(txt_basenames.keys())
print(f"\n✓ Matching pairs: {len(matching_pairs)}")

print(f"\n" + "="*60)
print("RECOMMENDATIONS")
print("="*60)

if images_without_text:
    print(f"\n {len(images_without_text)} images have no labels")


if text_without_images:
    print(f"\n {len(text_without_images)} text files have no images")



✓ Matching pairs: 704

RECOMMENDATIONS

 8 images have no labels

 131 text files have no images


In [14]:
images_without_text

{'X51005433492(1)',
 'X51005442384(1)',
 'X51005605333(1)',
 'X51005676539(1)',
 'X51005685355(2)',
 'X51005685357(2)',
 'X51007339118(1)',
 'X51007339647(1)'}

In [10]:
text_without_images

{'X51005361946(1)',
 'X51005442361(1)',
 'X51005447839(1)',
 'X51005447840(1)',
 'X51005447848(1)',
 'X51005447850(1)',
 'X51005447852(1)',
 'X51005568900(1)',
 'X51005587254(1)',
 'X51005587254(2)',
 'X51005587254(3)',
 'X51005587254(4)',
 'X51005587254(5)',
 'X51005587267(1)',
 'X51005587267(2)',
 'X51005587267(3)',
 'X51005587267(4)',
 'X51005587267(5)',
 'X51005605284(5)',
 'X51005605285(4)',
 'X51005605285(5)',
 'X51005605286(4)',
 'X51005605286(5)',
 'X51005663317(1)',
 'X51005677329(1)',
 'X51005677329(2)',
 'X51005677331(2)',
 'X51005677332(2)',
 'X51005677335(2)',
 'X51005705722(2)',
 'X51005705760(1)',
 'X51005705760(2)',
 'X51005705804(1)',
 'X51005705804(2)',
 'X51005719912(1)',
 'X51005719912(2)',
 'X51005719914(1)',
 'X51005719914(2)',
 'X51005719917(1)',
 'X51005719917(2)',
 'X51005722668(1)',
 'X51005722668(2)',
 'X51005722699(1)',
 'X51005757294(1)',
 'X51005757304(1)',
 'X51005757323(1)',
 'X51005757324(1)',
 'X51005757346(1)',
 'X51005806678(1)',
 'X51005806678(2)',


In [19]:
import os
import shutil
from pathlib import Path

def organize_data_files():
    parent_dir = os.path.dirname(data_dir)
    good_data_dir = os.path.join(parent_dir, 'good_data')
    bad_data_dir = os.path.join(parent_dir, 'bad_data')
    
    # Create output directories if they don't exist
    os.makedirs(good_data_dir, exist_ok=True)
    os.makedirs(bad_data_dir, exist_ok=True)
    
    # Get all files in the data directory
    all_files = os.listdir(data_dir)
    
    # Separate image and text files
    image_files = [f for f in all_files if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff'))]
    text_files = [f for f in all_files if f.lower().endswith('.txt')]
    
    # Create sets of base names (without extensions)
    image_basenames = {os.path.splitext(f)[0] for f in image_files}
    text_basenames = {os.path.splitext(f)[0] for f in text_files}
    
    # Find matching pairs and orphans
    matching_pairs = image_basenames & text_basenames
    orphan_images = image_basenames - text_basenames
    orphan_texts = text_basenames - image_basenames
    
    print(f"Total images: {len(image_files)}")
    print(f"Total text files: {len(text_files)}")
    print(f"Matching pairs: {len(matching_pairs)}")
    print(f"Orphan images: {len(orphan_images)}")
    print(f"Orphan text files: {len(orphan_texts)}")
    
    # Copy matching pairs to good_data directory
    for base_name in matching_pairs:
        # Find the actual files with extensions
        image_file = next((f for f in image_files if os.path.splitext(f)[0] == base_name), None)
        text_file = next((f for f in text_files if os.path.splitext(f)[0] == base_name), None)
        
        if image_file and text_file:
            shutil.copy2(os.path.join(data_dir, image_file), good_data_dir)
            shutil.copy2(os.path.join(data_dir, text_file), good_data_dir)
    
    # Copy orphan images to Bad_data directory
    for base_name in orphan_images:
        image_file = next((f for f in image_files if os.path.splitext(f)[0] == base_name), None)
        if image_file:
            shutil.copy2(os.path.join(data_dir, image_file), bad_data_dir)
    
    # Copy orphan text files to Bad_data directory
    for base_name in orphan_texts:
        text_file = next((f for f in text_files if os.path.splitext(f)[0] == base_name), None)
        if text_file:
            shutil.copy2(os.path.join(data_dir, text_file), bad_data_dir)
    
    # Verify the results
    good_images = len([f for f in os.listdir(good_data_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff'))])
    good_texts = len([f for f in os.listdir(good_data_dir) if f.lower().endswith('.txt')])
    bad_images = len([f for f in os.listdir(bad_data_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff'))])
    bad_texts = len([f for f in os.listdir(bad_data_dir) if f.lower().endswith('.txt')])
    
    print("\n=== Organization Complete ===")
    print(f"Good_data directory: {good_images} images, {good_texts} text files")
    print(f"Bad_data directory: {bad_images} images, {bad_texts} text files")
    
    return {
        'matching_pairs': len(matching_pairs),
        'orphan_images': len(orphan_images),
        'orphan_texts': len(orphan_texts)
    }

def clean_previous_runs():
    """Clean up previous runs to start fresh"""
    data_dir = '../Data/0325updated.task1train(626p)-20251119T175414Z-1-001/0325updated.task1train(626p)'
    good_data_dir = os.path.join(data_dir, 'good_data')
    bad_data_dir = os.path.join(data_dir, 'Bad_data')
    
    # Remove directories if they exist
    if os.path.exists(good_data_dir):
        shutil.rmtree(good_data_dir)
        print("Cleaned previous good_data directory")
    
    if os.path.exists(bad_data_dir):
        shutil.rmtree(bad_data_dir)
        print("Cleaned previous bad_data directory")

if __name__ == "__main__":
    # Clean previous runs first
    print("Cleaning previous runs...")
    clean_previous_runs()
    
    print("\nOrganizing files...")
    results = organize_data_files()
    
    print(f"\n=== FINAL RESULTS ===")
    print(f"Matching pairs: {results['matching_pairs']}")
    print(f"Orphan images: {results['orphan_images']}")
    print(f"Orphan text files: {results['orphan_texts']}")

Cleaning previous runs...

Organizing files...
Total images: 712
Total text files: 835
Matching pairs: 704
Orphan images: 8
Orphan text files: 131

=== Organization Complete ===
Good_data directory: 704 images, 704 text files
Bad_data directory: 8 images, 131 text files

=== FINAL RESULTS ===
Matching pairs: 704
Orphan images: 8
Orphan text files: 131


In [20]:
good_data_dir = os.path.join(os.path.dirname(data_dir),"good_data")

['X51005719862.jpg',
 'X51006329402.jpg',
 'X51006311764(1).txt',
 'X51005200938.txt',
 'X51006619862.jpg',
 'X51005605285.txt',
 'X51005719856.txt',
 'X51007103675.txt',
 'X51007103702.jpg',
 'X51006334926.txt',
 'X51006329399.jpg',
 'X51008164525.txt',
 'X51005442376(1).jpg',
 'X51006556828.txt',
 'X51006556838.jpg',
 'X51005719896.txt',
 'X51005441402.jpg',
 'X51006334926(3).txt',
 'X51007339098.jpg',
 'X51006619342.jpg',
 'X51008142068.txt',
 'X51006619862.txt',
 'X51006334926(1).jpg',
 'X51005677339(2).txt',
 'X51006556831.txt',
 'X51008164999.jpg',
 'X51008145450.txt',
 'X51008042782.jpg',
 'X51005444033.jpg',
 'X51005685357(1).jpg',
 'X51005442361.txt',
 'X51007339166.txt',
 'X51005447840.jpg',
 'X51006329395(2).jpg',
 'X51005724627.txt',
 'X51006557213.txt',
 'X51007846307.txt',
 'X51005711456.txt',
 'X51005433514.jpg',
 'X51007846301.txt',
 'X51005444045.jpg',
 'X51008128062.jpg',
 'X51007339157.jpg',
 'X51005442394.jpg',
 'X51005568913.jpg',
 'X51007339167(1).txt',
 'X5100567